# importing libs and setting video path


In [5]:
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import json
import os

video_path = 'videos/test.mp4'
output_path = 'videos/test_output.mp4'

# Intrusion Detection System  

## Polygon seting function

In [8]:
def set_polygon(video_path):
    capture = cv2.VideoCapture(video_path)
    if not capture.isOpened():
        print("Error opening video file")
        return []
    
    capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
    res, frame = capture.read()
    
    if not res:
        print("Error reading the first frame")
        return []   
    
    window_text = "SET POLYGONS - PRESS ESC TO FINISH"
    cv2.namedWindow(window_text, cv2.WINDOW_NORMAL)
    try:
        cv2.setWindowProperty(window_text, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    except Exception:
        pass

    display_frame = frame.copy()
    polygon_points = []

    def update_display():
        nonlocal display_frame, frame, polygon_points
        display_frame = frame.copy()

        instructions = [
            "INSTRUCTIONS:",
            "1. Click to add polygon vertices",
            "2. Press 'd' to delete last point",
            "3. Press 'c' to clear all polygon points",
            "4. Press ENTER when finished",
            "5. Press ESC to quit"
        ]

        for i, text in enumerate(instructions):
            cv2.putText(display_frame, text, (10, 30 + i*30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        if polygon_points:
            for i, (px, py) in enumerate(polygon_points):
                cv2.circle(display_frame, (px, py), 8, (0, 0, 255), -1)
                cv2.putText(display_frame, str(i+1), (px+15, py-15),
                           cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)
            if len(polygon_points) > 1:
                pts = np.array(polygon_points, dtype=np.int32)
                cv2.polylines(display_frame, [pts], True, (0, 255, 0), 2)
        
        cv2.imshow(window_text, display_frame)

    def mouse_callback(event, x, y, flags, param):
        nonlocal polygon_points
        if event == cv2.EVENT_LBUTTONDOWN:
            polygon_points.append((x, y))
            update_display()
    
    cv2.setMouseCallback(window_text, mouse_callback)

    update_display()
    
    while True:
        key = cv2.waitKey(0) & 0xFF
        
        if key == 13 or key == 10:  # ENTER to finish
            if len(polygon_points) >= 3:
                break
            else:
                print("Need at least 3 points for a polygon")
        elif key == ord('d'):  # Delete last point
            if polygon_points:
                polygon_points.pop()
                update_display()
        elif key == ord('c'):  # Clear
            polygon_points = []
            update_display()
        elif key == 27:  # ESC 
            polygon_points = []
            break
    
    capture.release()
    cv2.destroyAllWindows()
    return polygon_points

In [9]:
print("Polygon points setting...")
polygon_points = set_polygon(video_path)

if not polygon_points:
    print("Polygon setting cancelled or no points defined.")
    exit()

print(f"Polygon setted with {len(polygon_points)} points:")
print(polygon_points)

shape_polygon = Polygon(polygon_points)

def save_polygon(points, filename='restricted_zones.json'):
    if not points:
        print("No points to save.")
        return  
    coordinates = {"restricted_zone": points}
    with open(filename, 'w') as f:
        json.dump(coordinates, f, indent=4)
    print(f"Polygon coordinates saved to {filename}")

save_polygon(polygon_points)


Polygon points setting...
Polygon setted with 4 points:
[(1122, 467), (925, 448), (952, 719), (1123, 719)]
Polygon coordinates saved to restricted_zones.json
